In [1]:
import os, warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge

from xgboost import XGBRegressor
from catboost import CatBoostRegressor


In [10]:
csv_path = './data/train.csv'
n_folds = 4
min_train_frac = 0.5

lag_list      = (1,2,5,10,20,60,120,240,360,480)
roll_windows  = (5,20,60,120,240,360)
z_windows     = (60,120)          
ewma_spans    = (16,32)           
rcorr_windows = (20,60,120)
rcov_windows  = (20,60,120)
rcorr_topk    = 6                 

# PCA usage
use_pca_y1 = False               
use_pca_y2 = False                 
pca_components_y2 = 32


In [3]:
def _pick_top_variance_features(df, base_cols, k=6):
    variances = df[base_cols].var().sort_values(ascending=False)
    return variances.index[:min(k, len(variances))].tolist()

def make_features_full(df,
                       lag_list=(1,2,5,10,20,60,120,240,360,480),
                       roll_windows=(5,20,60,120,240,360),
                       z_windows=(60,120),
                       ewma_spans=(16,32),
                       rcorr_windows=(20,60,120),
                       rcov_windows=(20,60,120),
                       rcorr_topk=6):
    df = df.copy()
    base_cols = [c for c in df.columns if c not in ['time','Y1','Y2']]

    # Lags
    for lag in lag_list:
        for c in base_cols:
            df[f"{c}_lag{lag}"] = df[c].shift(lag)

    # First differences & simple returns (shifted)
    for c in base_cols:
        df[f"{c}_diff1"] = df[c].diff().shift(1)
        with np.errstate(divide='ignore', invalid='ignore'):
            df[f"{c}_ret1"] = ((df[c] / df[c].shift(1)) - 1.0).shift(1)

    # Rolling mean/std on raw and on diffs (shifted)
    for w in roll_windows:
        for c in base_cols:
            s = df[c].shift(1)
            df[f"{c}_rollmean{w}"] = s.rolling(w).mean()
            df[f"{c}_rollstd{w}"]  = s.rolling(w).std()
            d = df[f"{c}_diff1"]
            df[f"{c}_diff1_rollmean{w}"] = d.rolling(w).mean()
            df[f"{c}_diff1_rollstd{w}"]  = d.rolling(w).std()

    # Rolling z-scores (shifted)
    for w in z_windows:
        for c in base_cols:
            s = df[c].shift(1)
            mu = s.rolling(w).mean()
            sd = s.rolling(w).std()
            df[f"{c}_z{w}"] = (s - mu) / sd

    # EWMA means (shifted)
    for span in ewma_spans:
        for c in base_cols:
            s = df[c].shift(1)
            df[f"{c}_ewma{span}"] = s.ewm(span=span, adjust=False).mean()

    # Rolling correlations, covariances, and beta among top-variance features (shifted inputs)
    topv = _pick_top_variance_features(df, base_cols, k=rcorr_topk)
    for w in rcorr_windows:
        for i in range(len(topv)):
            for j in range(i+1, len(topv)):
                a, b = topv[i], topv[j]
                s1 = df[a].shift(1); s2 = df[b].shift(1)
                df[f"rcorr_{a}_{b}_w{w}"] = s1.rolling(w).corr(s2)

    for w in rcov_windows:
        for i in range(len(topv)):
            for j in range(i+1, len(topv)):
                a, b = topv[i], topv[j]
                s1 = df[a].shift(1); s2 = df[b].shift(1)
                cov = s1.rolling(w).cov(s2)
                var_b = s2.rolling(w).var()
                beta = cov / var_b
                df[f"rcov_{a}_{b}_w{w}"]  = cov
                df[f"rbeta_{a}_on_{b}_w{w}"] = beta

    # Drop NaNs created by lags/rolls
    df = df.dropna().reset_index(drop=True)

    feature_cols = [c for c in df.columns if c not in ['Y1','Y2']]
    X_all = df[feature_cols]
    y1_all = df['Y1']
    y2_all = df['Y2']
    return X_all, y1_all, y2_all

In [4]:
def build_folds_by_time(times, n_folds=4, min_train_frac=0.5):
    uniq = np.unique(times)
    n = len(uniq)
    n_folds = max(2, n_folds)
    start_train_idx = int(n * min_train_frac)
    remain = n - start_train_idx
    seg = max(1, remain // n_folds)
    folds = []
    for k in range(n_folds):
        val_start = start_train_idx + k * seg
        val_end   = start_train_idx + (k + 1) * seg if k < n_folds - 1 else n
        if val_start >= val_end or val_start >= n:
            continue
        train_time_max = uniq[val_start - 1]
        val_time_start = uniq[val_start]
        val_time_end   = uniq[val_end - 1]
        train_idx = np.where(times <= train_time_max)[0]
        val_mask  = (times >= val_time_start) & (times <= val_time_end)
        val_idx   = np.where(val_mask)[0]
        if len(train_idx) and len(val_idx):
            folds.append((train_idx, val_idx))
    return folds

In [ ]:
def models_y1():
    models = []
    if XGBRegressor is not None:
        models.append(('XGBoost', XGBRegressor(
            objective='reg:squarederror',
            learning_rate=0.02,
            n_estimators=3500,
            max_depth=5,
            subsample=0.9,
            colsample_bytree=0.9,
            reg_lambda=3.0,
            tree_method='hist',
            random_state=42,
            verbosity=0
        )))
    if CatBoostRegressor is not None:
        models.append(('CatBoost', CatBoostRegressor(
            loss_function='RMSE',
            learning_rate=0.03,
            depth=6,
            l2_leaf_reg=8.0,
            iterations=3000,
            random_seed=42,
            verbose=False
        )))
    return models

In [6]:
def run_cv(X, y, models, folds, use_pca=False, pca_components=32):
    results = {name: [] for name, _ in models}
    preds_cache = {name: [] for name, _ in models}  # list of (val_indices, preds) per fold

    for fold_id, (tr_idx, va_idx) in enumerate(folds, 1):
        X_tr = X.iloc[tr_idx].copy(); X_va = X.iloc[va_idx].copy()
        y_tr = y.iloc[tr_idx].copy(); y_va = y.iloc[va_idx].copy()
        feat_cols = [c for c in X_tr.columns if c != 'time']
        X_tr_f = X_tr[feat_cols].values; X_va_f = X_va[feat_cols].values

        if use_pca:
            scaler = StandardScaler(with_mean=True, with_std=True)
            X_tr_s = scaler.fit_transform(X_tr_f)
            X_va_s = scaler.transform(X_va_f)
            pca = PCA(n_components=min(pca_components, X_tr_s.shape[1]))
            X_tr_use = pca.fit_transform(X_tr_s)
            X_va_use = pca.transform(X_va_s)
        else:
            X_tr_use = X_tr_f
            X_va_use = X_va_f

        for name, model in models:
            model.fit(X_tr_use, y_tr)
            y_hat = model.predict(X_va_use)
            preds_cache[name].append((va_idx, y_hat))
            r2 = r2_score(y_va, y_hat)
            results[name].append(r2)

        print(f"[fold {fold_id}/{len(folds)}] done")

    return results, preds_cache

In [7]:
assert os.path.exists(csv_path), f"CSV not found: {csv_path}"
df = pd.read_csv(csv_path).sort_values('time').reset_index(drop=True)
print("Loaded:", df.shape)

X_all, y1_all, y2_all = make_features_full(
    df,
    lag_list=lag_list,
    roll_windows=roll_windows,
    z_windows=z_windows,
    ewma_spans=ewma_spans,
    rcorr_windows=rcorr_windows,
    rcov_windows=rcov_windows,
    rcorr_topk=rcorr_topk
)
print("Features shape:", X_all.shape)

times = X_all['time'].values
folds = build_folds_by_time(times, n_folds=n_folds, min_train_frac=min_train_frac)
print("Folds:", len(folds))

Loaded: (80000, 17)
Features shape: (79520, 710)
Folds: 4


In [8]:
mdl_y1 = models_y1()
if len(mdl_y1) == 0:
    raise RuntimeError("No Y1 models available (need at least XGBoost or CatBoost).")

res_y1, cache_y1 = run_cv(
    X_all, y1_all, mdl_y1, folds,
    use_pca=use_pca_y1, pca_components=0  # Y1 PCA off
)

# Build equal-weight blend if both are present; else use the single available model
names = [name for name, _ in mdl_y1]
if len(names) >= 2:
    blend_scores = []
    for i, (_, va_idx) in enumerate(folds):
        yhats = []
        for name in names:
            idxs, pred = cache_y1[name][i]
            yhats.append(pred)
        y_blend = np.mean(np.vstack(yhats), axis=0)
        r2 = r2_score(y1_all.iloc[va_idx], y_blend)
        blend_scores.append(r2)
    print("\n[Y1] Equal-weight blend (XGB+Cat):")
    print("Mean R²:", float(np.mean(blend_scores)))
    print("Std  R²:", float(np.std(blend_scores)))
    print("Folds :", [float(x) for x in blend_scores])
else:
    only = names[0]
    print("\n[Y1] Single model (no blend available):", only)
    print("Mean R²:", float(np.mean(res_y1[only])))
    print("Std  R²:", float(np.std(res_y1[only])))
    print("Folds :", [float(x) for x in res_y1[only]])

[fold 1/4] done
[fold 2/4] done
[fold 3/4] done
[fold 4/4] done

[Y1] Equal-weight blend (XGB+Cat):
Mean R²: 0.7423919326651036
Std  R²: 0.03567672216073825
Folds : [0.7722805893392132, 0.7797416230064587, 0.6923033147909978, 0.7252422035237454]


In [11]:
def y2_equal_weight_blend(cache, y, folds, names=("XGBoost","CatBoost")):
    if not all(n in cache for n in names):
        return None
    scores = []
    for i, (_, va_idx) in enumerate(folds):
        yhats = []
        for n in names:
            idxs, pred = cache[n][i]
            yhats.append(pred)
        y_blend = np.mean(np.vstack(yhats), axis=0)
        scores.append(r2_score(y.iloc[va_idx], y_blend))
    return scores

def best_convex_weight_1d(cache, y, folds, names=("XGBoost","CatBoost"), grid_step=0.01):
    # Build full OOF vectors
    X_xgb, X_cat, y_true = [], [], []
    for i, (_, va_idx) in enumerate(folds):
        _, px = cache[names[0]][i]
        _, pc = cache[names[1]][i]
        X_xgb.append(px); X_cat.append(pc); y_true.append(y.iloc[va_idx].values)
    X_xgb = np.concatenate(X_xgb)
    X_cat = np.concatenate(X_cat)
    y_true = np.concatenate(y_true)

    best_w, best_r2 = 0.5, -1e9
    ws = np.arange(0.0, 1.0 + 1e-9, grid_step)
    for w in ws:
        y_hat = w * X_xgb + (1 - w) * X_cat
        r2 = r2_score(y_true, y_hat)
        if r2 > best_r2:
            best_w, best_r2 = float(w), float(r2)
    return best_w, best_r2

def apply_convex_weight_per_fold(cache, y, folds, w, names=("XGBoost","CatBoost")):
    scores = []
    for i, (_, va_idx) in enumerate(folds):
        _, px = cache[names[0]][i]
        _, pc = cache[names[1]][i]
        y_hat = w * px + (1 - w) * pc
        scores.append(r2_score(y.iloc[va_idx], y_hat))
    return scores

In [12]:
mdl_y2 = models_y2()
if len(mdl_y2) == 0:
    raise RuntimeError("No Y2 models available (need at least XGBoost or CatBoost).")

res_y2, cache_y2 = run_cv(
    X_all, y2_all, mdl_y2, folds,
    use_pca=use_pca_y2, pca_components=pca_components_y2
)

names = [name for name, _ in mdl_y2]
if set(("XGBoost","CatBoost")).issubset(set(names)):
    # Show base-model CV first
    print("\n[Y2] Base models CV R²:")
    for n in ("XGBoost","CatBoost"):
        scores = res_y2[n]
        print(f"  {n:8s}: mean={np.mean(scores):.6f}, std={np.std(scores):.6f}, folds={np.round(scores,6)}")

    # Equal-weight blend
    eq_scores = y2_equal_weight_blend(cache_y2, y2_all, folds, ("XGBoost","CatBoost"))
    print("\n[Y2] Equal-weight blend (XGB+Cat):")
    print("  mean=", float(np.mean(eq_scores)), "std=", float(np.std(eq_scores)))
    print("  folds:", [float(s) for s in eq_scores])

    # Convex 1D blend (robust, avoids bad weights)
    w, r2_oof = best_convex_weight_1d(cache_y2, y2_all, folds, ("XGBoost","CatBoost"), grid_step=0.01)
    fold_scores = apply_convex_weight_per_fold(cache_y2, y2_all, folds, w, ("XGBoost","CatBoost"))
    print(f"\n[Y2] Convex blend (XGB weight={w:.2f}, Cat weight={1-w:.2f})")
    print("  OOF R² (global):", r2_oof)
    print("  CV folds R²     :", [float(s) for s in fold_scores])
    print("  mean=", float(np.mean(fold_scores)), "std=", float(np.std(fold_scores)))

else:
    # Only one model available
    only = names[0]
    print("\n[Y2] Single model (no blending):", only)
    print("  mean=", float(np.mean(res_y2[only])), "std=", float(np.std(res_y2[only])))
    print("  folds:", [float(x) for x in res_y2[only]])

[fold 1/4] done
[fold 2/4] done


KeyboardInterrupt: 

In [17]:
lag_list      = (1,2,5,10,20,60,120,240)     # no 360/480
roll_windows  = (5,20,60,120)                # no 240/360
rcorr_windows = (20,60,120)
rcov_windows  = (20,60,120)
rcorr_topk    = 6
use_pca_y2    = False                        # PCA OFF

In [18]:
def models_y2():
    models = []
    if XGBRegressor is not None:
        models.append(('XGBoost', XGBRegressor(
            objective='reg:squarederror',
            learning_rate=0.02,
            n_estimators=4000,
            max_depth=6,
            subsample=0.85,
            colsample_bytree=0.85,
            reg_lambda=4.0,
            tree_method='hist',
            random_state=42,
            verbosity=0
        )))
    if CatBoostRegressor is not None:
        models.append(('CatBoost', CatBoostRegressor(
            loss_function='RMSE',
            learning_rate=0.03,
            depth=7,
            l2_leaf_reg=10.0,
            iterations=3500,
            random_seed=42,
            verbose=False
        )))
    return models

In [19]:
# === Y2: train models on 80%, evaluate on 20% ===
mdl_y2 = models_y2()
names  = [name for name, _ in mdl_y2]
assert len(mdl_y2) > 0, "No Y2 models available."

preds = {}
for name, model in mdl_y2:
    # remove early stopping kwargs: fit on all train
    model.fit(X_tr_use, y_tr)
    preds[name] = model.predict(X_te_use)

# Report single-model R²
print("=== Y2 — 80/20 Holdout R² ===")
for name in preds:
    r2 = r2_score(y_te, preds[name])
    print(f"{name:8s}: R² = {r2:.6f}")

# Equal-weight blend (if both available)
if set(("XGBoost","CatBoost")).issubset(set(preds.keys())):
    y_eq = 0.5 * preds["XGBoost"] + 0.5 * preds["CatBoost"]
    print(f"{'Blend50/50':8s}: R² = {r2_score(y_te, y_eq):.6f}")

    # Best convex blend weight on test set
    ws = np.arange(0.0, 1.0 + 1e-9, 0.01)
    best_w, best_r2 = 0.5, -1e9
    for w in ws:
        y_blend = w * preds["XGBoost"] + (1 - w) * preds["CatBoost"]
        r2 = r2_score(y_te, y_blend)
        if r2 > best_r2:
            best_w, best_r2 = float(w), float(r2)
    print(f"Blend w*XGB+(1-w)*Cat (w={best_w:.2f}): R² = {best_r2:.6f}")
else:
    print("Blend: only one model available, skipping.")

=== Y2 — 80/20 Holdout R² ===
XGBoost : R² = 0.582033
CatBoost: R² = 0.555506
Blend50/50: R² = 0.571313
Blend w*XGB+(1-w)*Cat (w=1.00): R² = 0.582033
